In [28]:
import os
print(os.getcwd())

/home/jupyter/work/resources/yandex_metrika_cloud_case


In [5]:
import pandas as pd
import requests

# Загрузка данных в `ClickHouse`


In [16]:
hits_df = pd.read_csv('metrika_cloud_case_data_hits.csv', sep = '\t')
visits_df = pd.read_csv('metrika_cloud_case_data_visits.csv', sep = '\t')

## Подключение и настройка
https://cloud.yandex.ru/docs/managed-clickhouse/
(см. слайды)

## Данные для доступа
* Из интерфейса облака в разделе хосты копируем имя хост в переменную `CH_HOST_NAME`
* Используем заведенного юзера в переменной `CH_USER`
* Используем заведенного юзера в переменной `CH_USER`

* Сохраним пароль заведенного пользователя в текстовый файл `.chpass.txt`
* В переменную `CH_PASS` считаем содержимое файла `.chpass.txt`
* В переменную `cacert` поместим путь к сертификату для подключения к серверу. Файл `YandexInternalRootCA.crt` должен лежать в репозитории

In [3]:
#----------Вводить свои данные в нижние 3 переменные--------------
CH_HOST_NAME = 'rc1b-bbmw2gw58fm2dbeb.mdb.yandexcloud.net'
CH_USER      = 'z_sergey'
CH_DB_NAME   = 'metrica_data'

#-------------------------------------------
CH_PASS      = open('../.chpass.txt').read().strip()
CH_HOST      = f'https://{CH_HOST_NAME}:8443'
CH_CASERT    = 'YandexInternalRootCA.crt'

##  Функции для интеграции с ClickHouse

В файле `some_funcs` есть класс `simple_ch_client` для работы с ClickHouse

Сначала надо создать экземпляр класса, инициализировав его начальными параметрами - хост, пользователь, пароль и путь к сертификату
`simple_ch_client(CH_HOST, CH_USER, CH_PASS, cacert)`

В классе есть 4 метода:
* `.get_version()` - получает текущую версию ClickHouse. Хороший способ проверить, что указанные при инициализации параметры работают
* `.get_clickhouse_data(query)` - выполняет запрос `query` и возвращает результат в текстовом формате
* `.get_clickhouse_df(query)` - выполняет запрос `query` и возвращает результат в виде DataFrame
* `.upload(table, content)` - загружает таблицу `content`, которая подается в текстовом формате в таблицу ClickHouse'а с именем `table`


## Проверяем ClickHouse
Используя заговленные выше переменные проверим доступ до сервера (как в документации https://cloud.yandex.ru/docs/managed-clickhouse/operations/connect#connection-string)
Этот метод реализован в методе `.get_version()` класса для работы с ClickHouse
При успешном подключении не произойдет никакой ошибки при выполнении этого метода, и он сам вернет версию сервера ClickHouse (например `21.3.2.5`)

In [13]:
import some_funcs
from some_funcs import simple_ch_client

In [14]:
my_client = simple_ch_client(CH_HOST, CH_USER, CH_PASS, CH_CASERT)

In [15]:
my_client.get_version()

21.3.2.5



### Загружаем данные

#### Хиты

In [17]:
hits_df.head()

,ym:pv:browser,ym:pv:clientID,ym:pv:date,ym:pv:dateTime,ym:pv:deviceCategory,ym:pv:lastTrafficSource,ym:pv:operatingSystemRoot,ym:pv:URL
0,chrome,9625353358380463527,2020-07-06,2020-07-06 04:43:20,1,direct,macos,https://supermarket.ru/product_1280
1,chrome,9625353358380463527,2020-07-06,2020-07-06 04:43:34,1,direct,macos,https://supermarket.ru/product_1280
2,android_browser,15569343169762640095,2020-07-06,2020-07-06 13:26:04,2,referral,android,https://supermarket.ru/promo_action_2068
3,chrome,13994713805140870235,2020-07-06,2020-07-06 19:44:03,1,ad,macos,https://supermarket.ru/catalog_category_82
4,chrome,13994713805140870235,2020-07-06,2020-07-06 19:44:23,1,ad,macos,https://supermarket.ru/catalog_category_82


In [18]:
hits_df.rename(columns={'ym:pv:browser':'Browser',
                'ym:pv:clientID':'ClientID',
                'ym:pv:date':'EventDate',
                'ym:pv:dateTime':'EventTime',
                'ym:pv:deviceCategory':'DeviceCategory',
                'ym:pv:lastTrafficSource':'TraficSource',
                'ym:pv:operatingSystemRoot':'OSRoot',
                'ym:pv:URL':'URL'}, inplace = True)

In [19]:
q = f'drop table if exists {CH_DB_NAME}.hits '
my_client.get_clickhouse_data(q)

q = f'''
create table {CH_DB_NAME}.hits (
    Browser String,
    ClientID UInt64,
    EventDate Date,
    EventTime DateTime,
    DeviceCategory String,
    TraficSource String,
    OSRoot String,
    URL String
) ENGINE = MergeTree(EventDate, intHash32(ClientID), (EventDate, intHash32(ClientID)), 8192)
'''

my_client.get_clickhouse_data(q)

''

In [20]:
my_client.upload(
    f'{CH_DB_NAME}.hits',
    hits_df.to_csv(index = False, sep = '\t'))

''

#### Визиты

In [21]:
visits_df.head()

,ym:s:browser,ym:s:clientID,ym:s:date,ym:s:dateTime,ym:s:deviceCategory,ym:s:lastTrafficSource,ym:s:operatingSystemRoot,ym:s:purchaseID,ym:s:purchaseRevenue,ym:s:startURL
0,chrome,12267275798148517398,2020-09-18,2020-09-18 10:40:59,1,internal,windows,['16891366513471468610'],[10767],https://supermarket.ru/search
1,safari_mobile,10275295047121029551,2020-09-03,2020-09-03 22:55:32,2,organic,ios_double,[],[],https://supermarket.ru/product_1798
2,chrome,16050187229564839418,2020-08-08,2020-08-08 03:46:34,1,direct,gnu_linux,[],[],https://supermarket.ru/product_678
3,chrome,4985086536106220175,2020-08-24,2020-08-24 21:21:35,1,ad,windows,[],[],https://supermarket.ru/
4,miui,6500393564893995094,2020-09-24,2020-09-24 07:00:20,2,direct,android,[],[],https://supermarket.ru/promo_action_2160


In [22]:
visits_df.rename(columns={'ym:s:browser':'Browser',
                'ym:s:clientID':'ClientID',
                'ym:s:date':'StartDate',
                'ym:s:dateTime':'StartTime',
                'ym:s:deviceCategory':'DeviceCategory',
                'ym:s:lastTrafficSource':'TraficSource',
                'ym:s:operatingSystemRoot':'OSRoot',
                'ym:s:purchaseRevenue': 'Purchase.Revenue', 
                'ym:s:purchaseID': 'Purchase.ID',
                'ym:s:startURL':'StartURL'}, inplace = True)

In [23]:
visits_df['Purchases'] = visits_df['Purchase.Revenue'].map(lambda x: x.count(',') + 1 if x != '[]' else 0 )
visits_df['Revenue'] = visits_df['Purchase.Revenue'].map(lambda x: sum(map(int,x[1:-1].split(','))) if x != '[]' else 0)

In [24]:
visits_df.drop(columns=['Purchase.ID','Purchase.Revenue'], inplace=True)

In [25]:
q = f'drop table if exists {CH_DB_NAME}.visits '
my_client.get_clickhouse_data(q)

q = f'''
create table {CH_DB_NAME}.visits (
    Browser String,
    ClientID UInt64,
    StartDate Date,
    StartTime DateTime,
    DeviceCategory UInt8,
    TraficSource String,
    OSRoot String,
    Purchases Int32,
    Revenue Double,
    StartURL String
) ENGINE = MergeTree(StartDate, intHash32(ClientID), (StartDate, intHash32(ClientID)), 8192)
'''

my_client.get_clickhouse_data(q)

''

In [26]:
my_client.upload(
    f'{CH_DB_NAME}.visits',
    visits_df.to_csv(sep='\t', index =False))

''